In [ ]:
import numpy
from src.models.conv_model import build_model as build_conv_model
from src.data_loader.RNASeqLoader import RNASeqDataGenerator
from src.models.conv_model import correlation_coefficient_loss, pearson_r

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv1D, Dropout, Flatten, BatchNormalization, MaxPool1D, Activation
from tensorflow.keras.optimizers import RMSprop
import tensorflow as tf

def custom_model(dropout):
    model = Sequential()
    model.add(Input(shape=(101,4)))
    model.add(Conv1D(filters=128, kernel_size=32, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(dropout))
    model.add(Conv1D(filters=64, kernel_size=32, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(dropout))
    model.add(Conv1D(filters=64, kernel_size=32, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(dropout))
    model.add(MaxPool1D(pool_size=2, padding="same"))
    model.add(Conv1D(filters=32, kernel_size=32, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(dropout))
    model.add(Conv1D(filters=32, kernel_size=32, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(dropout))
    model.add(Conv1D(filters=32, kernel_size=32, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(dropout))
    model.add(MaxPool1D(pool_size=2, padding="same"))
    model.add(Flatten())
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = tf.keras.optimizers.RMSprop(decay=0.0005)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse', correlation_coefficient_loss, pearson_r])
    return model
    
    

In [ ]:
#model = build_conv_model(101, 4, 64, 15, [16], blocks=2, dil_rate=1, pooling_size=1, dropout=0.1)
model = custom_model(0.3)
import numpy
from src.models.conv_model import build_model as build_conv_model
from src.data_loader.RNASeqLoader import RNASeqDataGenerator
import tensorflow as tf
from src.evaluator.evaluator import Evaluator
from src.models.conv_model import correlation_coefficient_loss, pearson_r
import math
dependencies = {
    'correlation_coefficient_loss': correlation_coefficient_loss,
    'pearson_r': pearson_r
}
def cosJump(epoch):
    max_lrate = -3
    min_lrate = -5
    t_s = 10
    epoch = epoch%10
    lrate = min_lrate + 0.5*(max_lrate-min_lrate)*(1+math.cos(epoch*1.0/t_s * math.pi))
    return math.pow(10, lrate)
lrate = tf.keras.callbacks.LearningRateScheduler(cosJump, verbose=1)
#model = tf.keras.models.load_model("models/cDNA-ABE/model-20-0.5040.h5", custom_objects=dependencies)
optimizer = tf.keras.optimizers.RMSprop(lr=3e-5)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse', correlation_coefficient_loss, pearson_r])
print(model.summary())

In [ ]:
train_generator = RNASeqDataGenerator("data/processed/cDNA-CBE/train_data.hdf5", 1024, sqrt_edits=False)
validation_generator = RNASeqDataGenerator("data/processed/cDNA-CBE/validation_data.hdf5", 1024, sqrt_edits=False)

In [ ]:
import tensorflow as tf
from time import time

checkpoint_filepath = "models/cDNA-CBE/model-{epoch:02d}-{val_pearson_r:.4f}.h5"

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    #monitor='val_pearson_r',
    #mode='max',
    save_best_only=False)
history = model.fit(x=train_generator, epochs=300, validation_data=validation_generator, callbacks=[model_checkpoint_callback], use_multiprocessing=True, workers=40)

In [ ]:
import numpy
from src.models.conv_model import build_model as build_conv_model
from src.data_loader.RNASeqLoader import RNASeqDataGenerator

import tensorflow as tf
from src.evaluator.evaluator import Evaluator
from src.models.conv_model import correlation_coefficient_loss, pearson_r
test_generator = RNASeqDataGenerator("data/processed/cDNA-ABE/test_data.hdf5", 3102)
dependencies = {
    'correlation_coefficient_loss': correlation_coefficient_loss,
    'pearson_r': pearson_r
}

model = tf.keras.models.load_model("models/cDNA-ABE/model-25-0.5054.h5", custom_objects=dependencies)

ev = Evaluator(model, test_generator)
ev.evaluate()

ev.generatePlot("cDNA-ABE sequence")

In [4]:
import h5py
with h5py.File("data/processed/cDNA-ABE/test_structure_data.hdf5") as hf:
    print(hf['structures'][0])

b'))))((((((.......)))))).)))))))((((((((((.((...(((.(((.(((...))).))))))..........((......))((((((...('


/opt/miniconda3/envs/tf1.15.2/lib/python3.7/site-packages/ipykernel_launcher.py:2: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  
